In [1]:
import os
import torch
from torch import nn

torch.__version__

'2.0.0'

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [13]:
import os
import zipfile

from pathlib import Path
import random
import shutil
from shutil import copyfile

import requests
import pandas as pd

# Setup path to data folder
data_path = Path("/kaggle/input/ham10000-splitted/Images splitted/Splitted")
image_path = data_path 

In [15]:
classes = os.listdir(f'{image_path}/train')
classes_no_of_data_dict = {}
# classes.remove("desktop.ini")
classes

for i in range(len(classes)):
    classes_no_of_data_dict[classes[i]] = len(os.listdir(f'{image_path}/train/{classes[i]}'))
    
for i in range(len(classes)):
    temp = len(os.listdir(f'{image_path}/test/{classes[i]}'))
    classes_no_of_data_dict[classes[i]] = classes_no_of_data_dict[classes[i]] + temp

classes_no_of_data_dict

{'mel': 1113,
 'vasc': 142,
 'df': 115,
 'nv': 6705,
 'bkl': 1099,
 'akiec': 327,
 'bcc': 514}

In [22]:
# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('/kaggle/input/ham10000-splitted/Images splitted/Splitted/train'),
 PosixPath('/kaggle/input/ham10000-splitted/Images splitted/Splitted/test'))

In [23]:
from torchvision import datasets, transforms
from torchvision.transforms.functional import InterpolationMode

IMG_SIZE = 600

# Create transform pipeline manually   
data_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
#     transforms.RandomRotation((-120,120)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
#     transforms.RandomAffine(degrees=360, scale=(1.0, 1.3)),
#     transforms.RandomAdjustSharpness(sharpness_factor=0),
#     transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.RandomPerspective(distortion_scale=0.2),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    
    # Calculated for train data
    transforms.Normalize([0.5018, 0.5015, 0.5013], [0.1029, 0.0985, 0.0807]),
])     

print(f"Manually created transforms: {data_transform}")





data_transform_test = transforms.Compose([
#     transforms.CenterCrop(200),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
 
    # Calculated for test data
    transforms.Normalize([0.5018, 0.5015, 0.5013], [0.1029, 0.0985, 0.0807]),
])  


# Use ImageFolder to create dataset(s)
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)


test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform_test,
                                )

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

Manually created transforms: Compose(
    Resize(size=(600, 600), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    RandomHorizontalFlip(p=0.5)
    RandomVerticalFlip(p=0.5)
    RandomPerspective(p=0.5)
    Normalize(mean=[0.5018, 0.5015, 0.5013], std=[0.1029, 0.0985, 0.0807])
)
Train data:
Dataset ImageFolder
    Number of datapoints: 8017
    Root location: /kaggle/input/ham10000-splitted/Images splitted/Splitted/train
    StandardTransform
Transform: Compose(
               Resize(size=(600, 600), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               RandomPerspective(p=0.5)
               Normalize(mean=[0.5018, 0.5015, 0.5013], std=[0.1029, 0.0985, 0.0807])
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 1998
    Root location: /kaggle/input/ham10000-splitted/Images splitted/Splitted/test
    StandardTransform
T

In [25]:
# Get class names as a list
class_names = train_data.classes
class_names

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [26]:
# Can also get class names as a dict
class_dict = train_data.class_to_idx
class_dict

{'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}

In [28]:
# Check the lengths
len(train_data), len(test_data)

(8017, 1998)

In [30]:
# Turn train and test Datasets into DataLoaders
from torch.utils.data import DataLoader, WeightedRandomSampler
# from torchsampler import ImbalancedDatasetSampler

BATCH_SIZE = 32

# sampler = WeightedRandomSampler(sweights, 
#                                 num_samples=8017,
#                                 replacement=True)



train_dataloader = DataLoader(dataset=train_data, 
                              batch_size=BATCH_SIZE, # how many samples per batch?
                              num_workers=4, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True,
                              pin_memory=True,
#                               sampler=sampler, 
                              ) # shuffle the data?



test_dataloader = DataLoader(dataset=test_data, 
                             batch_size=BATCH_SIZE, 
                             num_workers=4, 
                             shuffle=False,
                             pin_memory=True,
                             ) # don't usually need to shuffle testing data


len(train_dataloader), len(test_dataloader)

(251, 63)

In [39]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential as Seq

# from gcn_lib import Grapher, act_layer

# for Kaggle import
import sys
sys.path.append("/kaggle/input/vig-pytorch/")
from gcn_lib import Grapher, act_layer


from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.models.helpers import load_pretrained
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from timm.models.registry import register_model


class Stem(nn.Module):
    """ Image to Visual Embedding
    Overlap: https://arxiv.org/pdf/2106.13797.pdf
    """
    def __init__(self, img_size=56, in_dim=3, out_dim=768, act='relu'):
        super().__init__()        
        self.convs = nn.Sequential(
            nn.Conv2d(in_dim, out_dim, 3, stride=1, padding=1),
            nn.BatchNorm2d(out_dim),
            act_layer(act),
            nn.Conv2d(out_dim, out_dim, 3, stride=1, padding=1),
            nn.BatchNorm2d(out_dim),
            act_layer(act),
            nn.Conv2d(out_dim, out_dim, 3, stride=1, padding=1),
            nn.BatchNorm2d(out_dim),
        )

    def forward(self, x):
        x = self.convs(x)
        return x
    
    
class Downsample(nn.Module):
    """ Convolution-based downsample
    """
    def __init__(self, in_dim=3, out_dim=768):
        super().__init__()        
        self.conv = nn.Sequential(
            nn.Conv2d(in_dim, out_dim, 3, stride=2, padding=1),
            nn.BatchNorm2d(out_dim),
        )

    def forward(self, x):
        x = self.conv(x)
        return x
    
    
    

class FFN(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act='relu', drop_path=0.0):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Sequential(
            nn.Conv2d(in_features, hidden_features, 1, stride=1, padding=0),
            nn.BatchNorm2d(hidden_features),
        )
        self.act = act_layer(act)
        self.fc2 = nn.Sequential(
            nn.Conv2d(hidden_features, out_features, 1, stride=1, padding=0),
            nn.BatchNorm2d(out_features),
        )
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        shortcut = x
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.drop_path(x) + shortcut
        return x#.reshape(B, C, N, 1)
    
    
    

class DeepGCN(torch.nn.Module):
    def __init__(self, opt):
        super(DeepGCN, self).__init__()
        print(opt)
        k = opt.k
        act = opt.act
        norm = opt.norm
        bias = opt.bias
        epsilon = opt.epsilon
        stochastic = opt.use_stochastic
        conv = opt.conv
        emb_dims = opt.emb_dims
        drop_path = opt.drop_path
        
        blocks = opt.blocks
        self.n_blocks = sum(blocks)
        channels = opt.channels
        reduce_ratios = [4, 2, 1, 1]
        dpr = [x.item() for x in torch.linspace(0, drop_path, self.n_blocks)]  # stochastic depth decay rule 
        num_knn = [int(x.item()) for x in torch.linspace(k, k, self.n_blocks)]  # number of knn's k
        max_dilation = 49 // max(num_knn)
        
        self.stem = Stem(out_dim=channels[0], act=act)
        self.pos_embed = nn.Parameter(torch.zeros(1, channels[0], 224//4, 224//4))
        HW = 224 // 4 * 224 // 4

        self.backbone = nn.ModuleList([])
        idx = 0
        for i in range(len(blocks)):
            if i > 0:
                self.backbone.append(Downsample(channels[i-1], channels[i]))
                HW = HW // 4
            for j in range(blocks[i]):
                self.backbone += [
                    Seq(Grapher(channels[i], num_knn[idx], min(idx // 4 + 1, max_dilation), conv, act, norm,
                                    bias, stochastic, epsilon, reduce_ratios[i], n=HW, drop_path=dpr[idx],
                                    relative_pos=True),
                          FFN(channels[i], channels[i] * 4, act=act, drop_path=dpr[idx])
                         )]
                idx += 1
        self.backbone = Seq(*self.backbone)

        self.prediction = Seq(nn.Conv2d(channels[-1], 1024, 1, bias=True),
                              nn.BatchNorm2d(1024),
                              act_layer(act),
                              nn.Dropout(opt.dropout),
                              nn.Conv2d(1024, opt.n_classes, 1, bias=True))
        self.model_init()

    def model_init(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
                m.weight.requires_grad = True
                if m.bias is not None:
                    m.bias.data.zero_()
                    m.bias.requires_grad = True

    def forward(self, inputs):
        x = self.stem(inputs) + self.pos_embed
        B, C, H, W = x.shape
        for i in range(len(self.backbone)):
            x = self.backbone[i](x)

        x = F.adaptive_avg_pool2d(x, 1)
        return self.prediction(x).squeeze(-1).squeeze(-1)

In [40]:
from going_modular.helper_functions import download_data, set_seeds, plot_loss_curves

In [41]:
def _cfg(url='', **kwargs):
    return {
        'url': url,
        'num_classes': 7, 'input_size': (3, 56, 56), 'pool_size': None,
        'crop_pct': .9, 'interpolation': 'bicubic',
        'mean': IMAGENET_DEFAULT_MEAN, 'std': IMAGENET_DEFAULT_STD,
        'first_conv': 'patch_embed.proj', 'classifier': 'head',
        **kwargs
    }


default_cfgs = {
    'vig_224_gelu': _cfg(
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5),
    ),
    'vig_b_224_gelu': _cfg(
        crop_pct=0.95, mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5),
    ),
}


def pvig_ti_224_gelu(pretrained=False, **kwargs):
    class OptInit:
        def __init__(self, num_classes=7, drop_path_rate=0.0, **kwargs):
            self.k = 9 # neighbor num (default:9)
            self.conv = 'mr' # graph conv layer {edge, mr}
            self.act = 'gelu' # activation layer {relu, prelu, leakyrelu, gelu, hswish}
            self.norm = 'batch' # batch or instance normalization {batch, instance}
            self.bias = False # bias of conv layer True or False
            self.dropout = 0.001 # dropout rate
            self.use_dilation = False # use dilated knn or not
            self.epsilon = 0.2 # stochastic epsilon for gcn
            self.use_stochastic = False # stochastic for gcn, True or False
            self.drop_path = drop_path_rate
            self.blocks = [2,2,6,2] # number of basic blocks in the backbone
            self.channels = [48, 96, 240, 384] # number of channels of deep features
            self.n_classes = num_classes # Dimension of out_channels
            self.emb_dims = 1024 # Dimension of embeddings

    opt = OptInit(**kwargs)
    model = DeepGCN(opt)
    model.default_cfg = default_cfgs['vig_224_gelu']
    return model



def pvig_b_224_gelu(pretrained=False, **kwargs):
    class OptInit:
        def __init__(self, num_classes=7, drop_path_rate=0.0, **kwargs):
            self.k = 9 # neighbor num (default:9)
            self.conv = 'mr' # graph conv layer {edge, mr}
            self.act = 'gelu' # activation layer {relu, prelu, leakyrelu, gelu, hswish}
            self.norm = 'batch' # batch or instance normalization {batch, instance}
            self.bias = False # bias of conv layer True or False
            self.dropout = 0.0 # dropout rate
            self.use_dilation = False # use dilated knn or not
            self.epsilon = 0.2 # stochastic epsilon for gcn
            self.use_stochastic = False # stochastic for gcn, True or False
            self.drop_path = drop_path_rate
            self.blocks = [2,2,18,2] # number of basic blocks in the backbone
            self.channels = [128, 256, 512, 1024] # number of channels of deep features
            self.n_classes = num_classes # Dimension of out_channels
            self.emb_dims = 1024 # Dimension of embeddings

    opt = OptInit(**kwargs)
    model = DeepGCN(opt)
    model.default_cfg = default_cfgs['vig_b_224_gelu']
    return model

In [43]:
model_fb = torch.hub.load('facebookresearch/swav:main', 'resnet50').to(device)

Downloading: "https://github.com/facebookresearch/swav/zipball/main" to /root/.cache/torch/hub/main.zip
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://dl.fbaipublicfiles.com/deepcluster/swav_800ep_pretrain.pth.tar" to /root/.cache/torch/hub/checkpoints/swav_800ep_pretrain.pth.tar
100%|██████████| 108M/108M [00:00<00:00, 211MB/s]  


In [44]:
# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Get the length of class_names (one output unit for each class)
# output_shape = len(class_names)

# Recreate the classifier layer and seed it to the target device
# model2.fc = torch.nn.Linear(in_features=1024, 
#                            out_features=7, # same number of output units as our number of classes
#                            bias=True).to(device)

In [46]:
from torchinfo import summary


model_summary = model_fb


summary(model=model_summary,
        input_size=(128, 3, 224, 224), # (batch_size, num_patches, embedding_dimension)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [128, 3, 224, 224]   [128, 1000]          --                   True
├─Conv2d (conv1)                         [128, 3, 224, 224]   [128, 64, 112, 112]  9,408                True
├─BatchNorm2d (bn1)                      [128, 64, 112, 112]  [128, 64, 112, 112]  128                  True
├─ReLU (relu)                            [128, 64, 112, 112]  [128, 64, 112, 112]  --                   --
├─MaxPool2d (maxpool)                    [128, 64, 112, 112]  [128, 64, 56, 56]    --                   --
├─Sequential (layer1)                    [128, 64, 56, 56]    [128, 256, 56, 56]   --                   True
│    └─Bottleneck (0)                    [128, 64, 56, 56]    [128, 256, 56, 56]   --                   True
│    │    └─Conv2d (conv1)               [128, 64, 56, 56]    [128, 64, 56, 56]    4,096                True
│    │    └─BatchN

In [47]:
model_fb.fc = torch.nn.Linear(in_features=2048, 
                           out_features=7, # same number of output units as our number of classes
                           bias=True).to(device)

In [48]:
# model1 = nn.DataParallel(model1)  ### for two GPU faster computations 

In [49]:
# model2 = nn.DataParallel(model2)  ### for two GPU faster computations 

In [50]:
model_fb = nn.DataParallel(model_fb)  ### for two GPU faster computations 

In [53]:
import wandb

# start a new wandb run to track this script
tracking = wandb.init(
    # set the wandb project where this run will be logged
    project="Glaucoma Fundus Imaging",
    name="HAM10000: Fb_model",
#     name="Origa: Fb_model",
    notes="",
    # track hyperparameters and run metadata
    config={
        "learning_rate": 2e-3,
        "architecture": "fb",
        "dataset": "Default",
        "epochs": 200,
    }
)

# 4b7dfb240ea0d5aae1afac2de518c0e940547396

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# **Training**

In [54]:
"""
Contains functions for training and testing a PyTorch model.
"""
from typing import Dict, List, Tuple
from sklearn.metrics import classification_report

import torch

from tqdm.auto import tqdm
import wandb


def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device,
               len_train_data) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss = 0.0 
    train_acc = 0.0

    # for evaluation
    y_true_train_data = []
    y_predicted_train_data = []

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)


        # 1. Forward pass
        y_pred = model(X)
        
#         y_pred = y_pred.logits

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        # train_acc += (y_pred_class == y).sum().item()/len(y_pred)
        train_acc += (y_pred_class == y).sum().item()
        
        # for evaluaiton
        y_true_train_data.append(y)
        y_predicted_train_data.append(y_pred_class)

    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
#     train_acc = train_acc / len(dataloader)
    train_acc = train_acc / len_train_data
    
    #     New way of accuracy calculation
    true = 0
    tot = 0
    for i in range(len(y_true_train_data)):
        x = (y_true_train_data[i] == y_predicted_train_data[i])
        for j in range(len(x)):
            tot += 1
            if x[j] == True:
                true += 1
    train_acc = (true/tot)

    # return train_loss, train_acc
    return train_loss, train_acc, y_true_train_data, y_predicted_train_data




def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device,
              len_test_data) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
    """
    # Put model in eval mode
    model.eval() 

    # Setup test loss and test accuracy values
    test_loss = 0.0 
    test_acc = 0.0

    # for evaluation
    y_true_test_data = []
    y_predicted_test_data = []


    # Turn on inference context manager
    with torch.inference_mode():

        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)
            
#             test_pred_logits = test_pred_logits.logits
            
            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            # test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
            test_acc += (test_pred_labels == y).sum().item()

            # test_pred_labels = torch.argmax(torch.softmax(test_pred_logits, dim=1), dim=1)
            # test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels)) 
            

            # for evaluaiton
            y_true_test_data.append(y)
            y_predicted_test_data.append(test_pred_labels)


    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
#     test_acc = test_acc / len(dataloader)
    test_acc = test_acc / len_test_data


    
#     New way of accuracy calculation
    true = 0
    tot = 0
    for i in range(len(y_true_test_data)):
        x = (y_true_test_data[i] == y_predicted_test_data[i])
        for j in range(len(x)):
            tot += 1
            if x[j] == True:
                true += 1
    test_acc = (true/tot)
    
    # return test_loss, test_acc
    return test_loss, test_acc, y_true_test_data, y_predicted_test_data




def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          tracking,
          best_accuracy,
          len_train_data,
          len_test_data,
        ) -> Dict[str, List[float]]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]} 
    For example if training for epochs=2: 
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }


        
    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc, y_true_train_data, y_predicted_train_data = train_step(model=model,
                                                                                      dataloader=train_dataloader,
                                                                                      loss_fn=loss_fn,
                                                                                      optimizer=optimizer,
                                                                                      device=device,
                                                                                      len_train_data=len_train_data,
                                                                                      )
        test_loss, test_acc, y_true_test_data, y_predicted_test_data = test_step(model=model,
                                                                                 dataloader=test_dataloader,
                                                                                 loss_fn=loss_fn,
                                                                                 device=device,
                                                                                 len_test_data=len_test_data,
                                                                                 )

        if best_accuracy < test_acc:
            print(f"\nAccuracy Improved ({best_accuracy} to {test_acc}), Saving the model...............................................\n")
            best_accuracy = test_acc
            best_y_true_test_data = y_true_test_data
            best_y_predicted_test_data = y_predicted_test_data

            model_name_path = f"{best_accuracy} acc.pth"
            torch.save(model.state_dict(), model_name_path)
            
            
                        
            # taking "y_predicted_train_data" & "y_true_train_data" into 1D array because it came out as batch by batch 2D list
            predicted_train_data_1D = []
            true_train_data_1D = []

            for i in range(len(y_predicted_train_data)):
                for j in range(len(y_predicted_train_data[i])):
                    predicted_train_data_1D.append(y_predicted_train_data[i][j])
                    true_train_data_1D.append(y_true_train_data[i][j])

            # taking both into CPU
            predicted_train_data_cpu = torch.tensor(predicted_train_data_1D, device = 'cpu')
            true_train_data_cpu = torch.tensor(true_train_data_1D, device = 'cpu')



            # now same procedure for test data's
            predicted_test_data_1D = []
            true_test_data_1D = []

            for i in range(len(y_predicted_test_data)):
                for j in range(len(y_predicted_test_data[i])):
                    predicted_test_data_1D.append(y_predicted_test_data[i][j])
                    true_test_data_1D.append(y_true_test_data[i][j])

            # taking both into CPU
            predicted_test_data_cpu = torch.tensor(predicted_test_data_1D, device = 'cpu')
            true_test_data_cpu = torch.tensor(true_test_data_1D, device = 'cpu')



            # now same procedure for best test data's
            best_predicted_test_data_1D = []
            best_true_test_data_1D = []

            for i in range(len(best_y_predicted_test_data)):
                for j in range(len(best_y_predicted_test_data[i])):
                    best_predicted_test_data_1D.append(best_y_predicted_test_data[i][j])
                    best_true_test_data_1D.append(best_y_true_test_data[i][j])

            # taking both into CPU
            best_predicted_test_data_cpu = torch.tensor(best_predicted_test_data_1D, device = 'cpu')
            best_true_test_data_cpu = torch.tensor(best_true_test_data_1D, device = 'cpu')
            
            

            # Generate a classification report
            # F1 score on train data
            y_true_train = true_train_data_cpu
            y_pred_train = predicted_train_data_cpu

            report = classification_report(y_true_train, y_pred_train, target_names=class_names)

            print(f"Evaluation report on Train data: \n\n{report}\n\n\n\n")


            # F1 score on test data
            y_true_test = true_test_data_cpu
            y_pred_test = predicted_test_data_cpu

            report = classification_report(y_true_test, y_pred_test, target_names=class_names)

            print(f"Evaluation report on Test data: \n\n{report}")
            
            
            
            
            
            
        #     Live result tracking#####################################
        #     for i in range(len(y_true_test_data)):
        #         print(f"{y_true_test_data[i]} and {y_predicted_test_data[i]}")
        acc_0 = 0
        acc_1 = 0
        total_0 = 0
        total_1 = 0
        for i in range(len(y_predicted_test_data)):
            for j in range(len(y_true_test_data[i])):
                if (y_true_test_data[i][j] == 0):
                    total_0 += 1
                    if (y_true_test_data[i][j] == y_predicted_test_data[i][j]):
                        acc_0 += 1
                elif (y_true_test_data[i][j] == 1): 
                    total_1 += 1
                    if (y_true_test_data[i][j] == y_predicted_test_data[i][j]):
                        acc_1 += 1
#         print(f"\n0: {acc_0}/{total_0} and 1: {acc_1}/{total_1}\n\n")
        #     Live result tracking#####################################

        
        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f} | "
#           f"0: {acc_0}/{total_0} | "
#           f"1: {acc_1}/{total_1}"
        )
        

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        
        tracking.log({
            "train_acc": train_acc,
            "test_acc": test_acc,
        })

    # Return the filled results at the end of the epochs
    # return results
    

    return results, y_true_train_data, y_predicted_train_data, y_true_test_data, y_predicted_test_data, best_accuracy, best_y_true_test_data, best_y_predicted_test_data


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [55]:
from going_modular import engine

# Setup the optimizer to optimize our ViT model parameters using hyperparameters from the ViT paper 
optimizer = torch.optim.AdamW(params=model_fb.parameters(), 
#                               lr=2e-3, # Base LR from Table 3 for ViT-* ImageNet-1k                                    ## learning rate decre..
                              lr = 1e-3,
                              betas=(0.9, 0.999), # default values but also mentioned in ViT paper section 4.1 (Training & Fine-tuning)
                              weight_decay=0.05,
#                               weight_decay=1e-5
                              ) # from the ViT paper section 4.1 (Training & Fine-tuning) and Table 3 for ViT-* ImageNet-1k

# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=5)
# Setup the loss function for multi-class classification
loss_fn = torch.nn.CrossEntropyLoss()

# Set the seeds
set_seeds()
torch.manual_seed(42)
torch.cuda.manual_seed(42)

best_accuracy=0.0

In [ ]:
# Train the model and save the training results to a dictionary
results, y_true_train_data, y_predicted_train_data, y_true_test_data, y_predicted_test_data, best_accuracy, best_y_true_test_data, best_y_predicted_test_data = train(model=model_fb,
                                                                                                           train_dataloader=train_dataloader,
                                                                                                           test_dataloader=test_dataloader,
                                                                                                           optimizer=optimizer,
                                                                                                           loss_fn=loss_fn,
                                                                                                           epochs=300,
                                                                                                           device=device,
                                                                                                           tracking=tracking,
                                                                                                           best_accuracy=best_accuracy,
                                                                                                           len_train_data=len(train_data),
                                                                                                           len_test_data=len(test_data)
                                                                                                           )

  0%|          | 0/300 [00:00<?, ?it/s]


Accuracy Improved (0.0 to 0.7397397397397397), Saving the model...............................................

Evaluation report on Train data: 

              precision    recall  f1-score   support

       akiec       0.39      0.19      0.25       262
         bcc       0.37      0.29      0.33       412
         bkl       0.46      0.36      0.40       880
          df       0.00      0.00      0.00        93
         mel       0.52      0.26      0.35       891
          nv       0.79      0.94      0.86      5365
        vasc       0.00      0.00      0.00       114

    accuracy                           0.72      8017
   macro avg       0.36      0.29      0.31      8017
weighted avg       0.67      0.72      0.68      8017





Evaluation report on Test data: 

              precision    recall  f1-score   support

       akiec       0.86      0.09      0.17        65
         bcc       0.49      0.41      0.45       102
         bkl       0.60      0.45      0.52       219


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classif


Accuracy Improved (0.7397397397397397 to 0.7727727727727728), Saving the model...............................................

Evaluation report on Train data: 

              precision    recall  f1-score   support

       akiec       0.46      0.34      0.39       262
         bcc       0.51      0.47      0.49       412
         bkl       0.55      0.49      0.52       880
          df       0.00      0.00      0.00        93
         mel       0.52      0.31      0.39       891
          nv       0.83      0.94      0.88      5365
        vasc       0.65      0.27      0.38       114

    accuracy                           0.76      8017
   macro avg       0.50      0.40      0.44      8017
weighted avg       0.72      0.76      0.73      8017





Evaluation report on Test data: 

              precision    recall  f1-score   support

       akiec       1.00      0.05      0.09        65
         bcc       0.46      0.62      0.53       102
         bkl       0.71      0.50      

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classif


Accuracy Improved (0.7727727727727728 to 0.7897897897897898), Saving the model...............................................

Evaluation report on Train data: 

              precision    recall  f1-score   support

       akiec       0.57      0.42      0.48       262
         bcc       0.57      0.63      0.60       412
         bkl       0.61      0.55      0.58       880
          df       0.33      0.01      0.02        93
         mel       0.60      0.39      0.47       891
          nv       0.86      0.94      0.90      5365
        vasc       0.73      0.54      0.62       114

    accuracy                           0.79      8017
   macro avg       0.61      0.50      0.52      8017
weighted avg       0.77      0.79      0.77      8017





Evaluation report on Test data: 

              precision    recall  f1-score   support

       akiec       0.35      0.88      0.50        65
         bcc       0.78      0.44      0.56       102
         bkl       0.75      0.47      

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Accuracy Improved (0.7897897897897898 to 0.8143143143143143), Saving the model...............................................

Evaluation report on Train data: 

              precision    recall  f1-score   support

       akiec       0.59      0.44      0.50       262
         bcc       0.61      0.68      0.64       412
         bkl       0.64      0.60      0.62       880
          df       0.29      0.02      0.04        93
         mel       0.62      0.43      0.51       891
          nv       0.87      0.94      0.91      5365
        vasc       0.78      0.68      0.72       114

    accuracy                           0.80      8017
   macro avg       0.63      0.54      0.56      8017
weighted avg       0.79      0.80      0.79      8017





Evaluation report on Test data: 

              precision    recall  f1-score   support

       akiec       0.42      0.82      0.56        65
         bcc       0.82      0.58      0.68       102
         bkl       0.78      0.48      